In [1]:
from utils import transliterate, is_alpha, load_SET_dataset, load_twitter_dataset, get_N_tokens, LABELS, read_and_split_file
import pandas as pd
import tempfile
N = 5000

def load_tails():
    interim_dir = "/home/peterr/macocu/taskB/data/interim"
    texts, labels = list(), list()
    files = ["bswac_tail_pp", "cnrwac_tail_pp_corrected_2", "hrwac_tail_pp", "srwac_tail_pp"]
    langs = ["bs", "me", "hr", "sr"]

    for file, lang in zip(files, langs):
        full_path = os.path.join(interim_dir, file)
        current_texts = read_and_split_file(full_path)
        len_cur_texts = len(current_texts)
        texts.extend(current_texts)
        labels.extend([lang] * len_cur_texts)

    return pd.DataFrame(data={"text": texts, "labels": labels})

train = load_tails()
#tokens = get_N_tokens(N=1000)
SET = load_SET_dataset().rename(columns={"language":"labels"})

dev_df = SET.loc[SET.split == "train", ["text", "labels"]]
test_df = SET.loc[SET.split != "train", ["text", "labels"]]


Let's prepare content in fasttext format that will be written to a file

In [2]:
content = ""
for i, row in train.iterrows():
    corrected_text = row['text'].replace('\n', ' ')
    content += f"""__label__{row['labels']} {corrected_text}"""

Let's prepare data in fasttext format in a temporary file:

In [16]:
import tempfile
  
fo = tempfile.NamedTemporaryFile()
print(fo.name)
with open(fo.name, "w") as f:
    f.write(content)

import fasttext

model = fasttext.train_supervised(fo.name,
                                epoch=50,
                                minn=3,
                                maxn=6
                                 )
fo.close()

/tmp/tmpq5rwnzua


Let's evaluate the model:

In [18]:

from sklearn.metrics import (
    f1_score,
    ConfusionMatrixDisplay,
    confusion_matrix,
    accuracy_score,
)
y_true = dev_df.labels
y_pred = [model.predict(s.replace("\n", " "))[0][0] for s in dev_df.text]
y_pred = [s[9:] for s in y_pred]
macro = f1_score(y_true, y_pred, labels=LABELS, average="macro")
micro = f1_score(y_true, y_pred, labels=LABELS, average="micro")
acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred, labels=LABELS)
returndict = dict(
    N=N,
    macroF1=macro,
    microF1=micro,
    accuracy=acc,
    cm = cm,

)
print(returndict)

{'N': 5000, 'macroF1': 0.0, 'microF1': 0.0, 'accuracy': 0.0, 'cm': array([[   0,    0,    0, 2556],
       [   0,    0,    0, 2556],
       [   0,    0,    0, 2262],
       [   0,    0,    0,    0]])}


In [12]:
model.predict(dev_df.text[0].replace("\n", " "))[0][0]

'__label__me'

In [14]:
y_pred

['me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',
 'me',

In [15]:
content[0:100]

'__label__bs   blizinu i povezanost sa klijentima kontinuirano graditi zadovoljstvo sigurnost i povje'

In [19]:
train.labels.value_counts()

me    140736
bs     75637
hr     62433
sr     52919
Name: labels, dtype: int64